In [1]:
import pybullet as p
import time
import pybullet_data
import math

# --- Helper Function to Visualize Orientation ---

def draw_frame(link_world_position, link_world_orientation):
    """Draws a coordinate frame (X,Y,Z) for a link to show its orientation."""
    line_length = 0.15
    # This function draws 3 lines for the X, Y, and Z axes of the link
    # The lines are given a short lifeTime so they flicker, which is less distracting
    # Colors: X=Red, Y=Green, Z=Blue
    p.addUserDebugLine(link_world_position, p.multiplyTransforms(link_world_position, link_world_orientation, [line_length, 0, 0], [0,0,0,1])[0], [1, 0, 0], 2, lifeTime=0.1)
    p.addUserDebugLine(link_world_position, p.multiplyTransforms(link_world_position, link_world_orientation, [0, line_length, 0], [0,0,0,1])[0], [0, 1, 0], 2, lifeTime=0.1)
    p.addUserDebugLine(link_world_position, p.multiplyTransforms(link_world_position, link_world_orientation, [0, 0, line_length], [0,0,0,1])[0], [0, 0, 1], 2, lifeTime=0.1)

# =====================================================================================
# --- Main Script Starts Here ---
# =====================================================================================

physicsClient = None # Initialize variable to hold the physics client

try:
    # --- 1. Connect to Physics Server (with defensive checking) ---
    print("Starting PyBullet simulation...")
    # This single block now handles connecting to the GUI.
    # The second, duplicated connection block has been removed.
    physicsClient = p.connect(p.GUI)
    print("Successfully connected to PyBullet GUI.")
    
    # --- 2. Configure Simulation ---
    p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0) # Tidy up the interface
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.81)
    p.setRealTimeSimulation(0) # We will step the simulation manually

    # --- 3. Load Models ---
    planeId = p.loadURDF("plane.urdf")
    tableId = p.loadURDF("table/table.urdf", basePosition=[0, -0.2, 0], useFixedBase=True)

    # Load Your Kinova Robot from the local path
    robot_path = "kinova_description/urdf/j2s6s300.urdf"
    robot_start_pos = [0, 0, 0.63]
    robot_start_orn = p.getQuaternionFromEuler([0, 0, 0])
    
    print(f"Loading robot from: {robot_path}")
    robotId = p.loadURDF(robot_path, robot_start_pos, robot_start_orn, useFixedBase=True)

    # Load objects to interact with
    duckId = p.loadURDF("duck_vhacd.urdf", basePosition=[0.4, -0.2, 0.7], globalScaling=0.8)
    cubeId = p.loadURDF("cube_small.urdf", basePosition=[0.3, -0.5, 0.7])

    # --- 4. Analyze Robot and Setup Interactive Controls ---
    num_joints = p.getNumJoints(robotId)
    controllable_joints = []
    end_effector_link_index = -1

    for i in range(num_joints):
        info = p.getJointInfo(robotId, i)
        joint_name = info[1].decode('utf-8')
        if info[2] == p.JOINT_REVOLUTE:
            controllable_joints.append(i)
        if joint_name == "j2s6s300_end_effector":
            end_effector_link_index = i

    if end_effector_link_index == -1:
        print("\nWarning: Could not find link 'j2s6s300_end_effector'. Using last link as default.")
        end_effector_link_index = num_joints - 1

    # Create a GUI slider for each controllable joint
    joint_sliders = []
    for joint_index in controllable_joints:
        info = p.getJointInfo(robotId, joint_index)
        joint_name = info[1].decode('utf-8')
        slider = p.addUserDebugParameter(paramName=joint_name, rangeMin=info[8], rangeMax=info[9], startValue=0)
        joint_sliders.append(slider)

    # This text will display the end-effector's pose
    text_id = p.addUserDebugText("End-Effector Pose", [0, -0.5, 1.5])

    # --- 5. Main Simulation Loop ---
    print("\nSimulation running... Move the sliders to control the robot.")
    while p.isConnected():
        target_positions = [p.readUserDebugParameter(slider) for slider in joint_sliders]

        p.setJointMotorControlArray(
            bodyUniqueId=robotId,
            jointIndices=controllable_joints,
            controlMode=p.POSITION_CONTROL,
            targetPositions=target_positions
        )

        link_state = p.getLinkState(robotId, end_effector_link_index)
        pos, orn = link_state[0], link_state[1]
        
        draw_frame(pos, orn)

        pos_str = f"Position (xyz): {pos[0]:.3f}, {pos[1]:.3f}, {pos[2]:.3f}"
        orn_euler = p.getEulerFromQuaternion(orn)
        orn_str = f"Orient (rpy): {orn_euler[0]:.3f}, {orn_euler[1]:.3f}, {orn_euler[2]:.3f}"
        
        text_id = p.addUserDebugText(f"{pos_str}\n{orn_str}", [0, -0.5, 1.5], 
                                     textColorRGB=[1, 1, 1], textSize=1.2,
                                     replaceItemUniqueId=text_id)

        p.stepSimulation()
        time.sleep(1./240.)

except KeyboardInterrupt:
    print("\nSimulation stopped by user.")
except p.error as e:
    # This will catch errors during the simulation, like failing to load a file.
    print(f"\nA PyBullet error occurred: {e}")
finally:
    # This 'finally' block ensures that we disconnect cleanly, even if an error occurs.
    if p.isConnected(physicsClient):
        print("Disconnecting from PyBullet.")
        p.disconnect()

Starting PyBullet simulation...
Successfully connected to PyBullet GUI.
Loading robot from: kinova_description/urdf/j2s6s300.urdf

A PyBullet error occurred: Cannot load URDF file.
Disconnecting from PyBullet.
